# Introdução ao Lasso e à regressão logística com regularização  $\ell_1$

Por Paulo J. S. Silva

## Lasso

O problema Lasso tem a forma

$$
 \min_x \frac{1}{2}\Vert Ax - b \Vert_2^2 + \lambda \Vert x \Vert_1.
$$

* Primeiro termo busca ajustar os dados por quadrados mínimos;
* O segundo termo (regularização $\ell_1$) busca soluções esparsas;
* O $\lambda$ pondera entre esses dois objetivos.

Combina ajuste com seleção de variáveis (características).


## Características

* Primeiro termo:
    * Diferenciável e tem estrutura simples;
    * Convexo (mais sobre isso depois);
    * Admite solução fechada (resolução de um sistema linear);
    * Estrutura simples que pode ser explorada de diversas formas;
    
* Segundo termo:
    * Convexo mas não diferenciável;
    * Dificulta usar métodos tradicionais;
    * Mas uma vez estrutura rica que admite diversas abordagens.

## Regressão logística com regularização $\ell_1$

O problema é

$$
\min_x \sum_{i = 1}^{n} \log (1 + \exp(-b_i A_{i, :} x)) + \lambda \Vert x \Vert_1,
$$

* Adequado para problemas de classificação $b = \pm 1$;
* Primeiro termo ainda diferenciável, porém mais "complicado";
* Mesmo segundo termo: não diferenciável com estrutura favorável.

## Dados para testes computacionais

* Os dois problemas serão usados de forma repetida como exemplos durante o curso;
* Serão usados para testar diferentes métodos;
* Não "gosto" de instâncias aleatórias.

[Dados](https://drive.google.com/drive/folders/1MXlxmYR4lqFPH5i-FTnCW1bAU2BeCqTp?usp=sharing) usados em Ronaldo Lopes, Sandra Santos, and Paulo J. S. Silva. ["Accelerating block coordinate
descent methods with identification strategies",Computational Optimization and Applications, 2019.](http://www.ime.unicamp.br/~pjssilva/papers/block_coordinate/)


In [4]:
# Inicializa projeto
import Pkg
Pkg.activate("../../code/")

# Localização dos dados
lassodir = "../../lasso-data/Data-Lasso"

# Importa código
include("mt853.jl")

data = readlasso(lassodir * "/SC10.mat")

  Activating project at `~/documentos/disciplinas/convex-alg/code`


LinearL1Problem(sparse([36, 120, 86, 52, 290, 268, 1, 256, 74, 125  …  89, 175, 114, 23, 222, 268, 76, 123, 200, 268], [1, 2, 3, 4, 5, 6, 7, 8, 9, 9  …  7844, 7844, 7845, 7846, 7846, 7846, 7847, 7847, 7847, 7847], [72.0, 427.0, 342.0, 116.0, 110.0, 67.0, 113.0, 595.0, 81.0, 213.0  …  42.0, 189.0, 75.0, 76.0, 66.0, 109.0, 141.0, 107.0, 57.0, 117.0], 300, 7847), [1.0, 1.0, -1.0, 1.0, -1.0, -1.0, -1.0, -1.0, 1.0, 1.0  …  1.0, -1.0, 1.0, -1.0, -1.0, 1.0, 1.0, 1.0, -1.0, 1.0], 1433.5, 114.7)

In [5]:
# Dimensões

@show size(data.A)
@show nnz(data.A);

size(data.A) = (300, 7847)
nnz(data.A) = 27949


In [6]:
# Há problemas enormes

data = readlasso(lassodir * "/SC17.mat")
println("SC17")
@show size(data.A)
@show nnz(data.A)
println()

data = readlasso(lassodir * "/SR18.mat")
println("SR18")
@show size(data.A)
@show nnz(data.A);


SC17
size(data.A) = (15564, 36842)
nnz(data.A) = 1028284

SR18
size(data.A) = (518571, 41400)
nnz(data.A) = 33486015


In [7]:
# E o que mais tem?

@show data.λ

@show data.ftarget;

data.λ = 11813.48203431546
data.ftarget = 3.473e7


In [9]:
# E os problemas de regressão logística?
logregdir = "../../lasso-data/Data-Logistic"

vars = matread(logregdir * "/SRlog17.mat")

Dict{String, Any} with 4 entries:
  "A"          => sparse([11082, 11204, 34989, 39570, 46481, 61048, 79167, 8269…
  "flogtarget" => 354600.0
  "b"          => [-1.0; -1.0; … ; 1.0; -1.0;;]
  "lambdalog"  => 507.577

In [6]:
data = readlogreg(logregdir * "/SRlog17.mat")
println("SRlog17")
@show size(data.A)
@show nnz(data.A)
@show data.λ, data.ftarget
@show data.b[1:5], data.b[end-5:end];

SRlog17
size(data.A) = (677399, 42735)
nnz(data.A) = 49556258
(data.λ, data.ftarget) = (507.5773072770321, 354600.0)
(data.b[1:5], data.b[end - 5:end]) = ([-1.0, -1.0, -1.0, -1.0, -1.0], [1.0, -1.0, -1.0, -1.0, 1.0, -1.0])


## Mas o que é um problema de otimização?

Dada uma função $f: \mathbb{R^n} \rightarrow \mathbb{R} \cup \{ \pm \infty \}$, o problema

$$
\min_{x \in \mathbb{R}^n} f(x)
$$

é uma notação para

Def. Encontre $x^* \in \mathbb{R}^n$, se existir, tal que
$$
f(x^*) = \inf \{ f(x) \mid x \in \mathbb{R}^n \} \in \mathbb{R} \cup \{ \pm \infty \}.
$$

Note que o ínfimo sempre existe e é chamado de *valor ótimo do problema*, já $x^*$ pode ou não existir e é chamado se *solução (ótima) ou (ponto de) ótimo* do problema de otimização.  

## E um problema de otimização com restrições?

Dada uma função $f: \mathbb{R^n} \rightarrow \mathbb{R} \cup \{ \pm \infty \}$ e $X \subset \mathbb{R}^n$, o problema

$$
\min_{x \in X} f(x)
$$

é uma notação para

Def. Encontre $x^* \in X$, se existir, tal que
$$
f(x^*) = \inf \{ f(x) \mid x \in X \} \in \mathbb{R} \cup \{ \pm \infty \}.
$$

Note que o ínfimo sempre existe e é chamado de *valor ótimo do problema*, já $x^*$ pode ou não existir e é chamado se *solução (ótima) ou (ponto de) ótimo* do problema de otimização.  